In [6]:
from pydub import AudioSegment
import os

orig_folder = "bird_voices"
mod_folder = "bird_voices_mod"
f_format = "mp3"
mp3_files = [f for f in os.listdir(orig_folder) if f.lower().endswith(".mp3")]

for mp3_file in mp3_files:

    f = AudioSegment.from_file(
        os.path.join(orig_folder, mp3_file),
        f_format
    )

    # оставляем часть файла с 3 секунды без голоса диктора
    f = f[3000:]

    f.export(
        os.path.join(mod_folder, mp3_file),
        f_format
    )

print("Успешная обрезка аудио")

Успешная обрезка аудио


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tqdm import tqdm

# === Настройки ===
input_dir = "bird_voices_mod"
output_dir = "mel_data"
segment_length = 4.0        # 4 секунды
overlap = 0.5               # 50%
sample_rate = 16000
n_mels = 64

os.makedirs(output_dir, exist_ok=True)

def process_audio(file_path, label):
    # --- Загружаем аудио ---
    y, sr = librosa.load(file_path, sr=sample_rate, mono=True)

    # --- Нормализация громкости ---
    y = librosa.util.normalize(y)

    # --- Параметры сегментации ---
    step = int(segment_length * sample_rate * (1 - overlap))  # шаг сдвига
    seg_len = int(segment_length * sample_rate)

    total_len = len(y)
    if total_len < seg_len:
        y = np.pad(y, (0, seg_len - total_len))  # добиваем до 4 секунд

    num_segments = max(1, (total_len - seg_len) // step + 1)

    # --- Папка для текущей птицы ---
    label_dir = os.path.join(output_dir, label)
    os.makedirs(label_dir, exist_ok=True)

    # --- Преобразование в мел-спектрограммы ---
    for i in range(num_segments):
        start = i * step
        end = start + seg_len
        segment = y[start:end]

        # Мел-спектрограмма
        mel = librosa.feature.melspectrogram(
            y=segment,
            sr=sample_rate,
            n_mels=n_mels,
            fmax=8000
        )

        mel_db = librosa.power_to_db(mel, ref=np.max)

        # Нормализация
        mel_db = (mel_db - np.mean(mel_db)) / (np.std(mel_db) + 1e-6)

        # --- Сохраняем ---
        base = os.path.splitext(os.path.basename(file_path))[0]
        out_path = os.path.join(label_dir, f"{base}_{i}.npy")
        np.save(out_path, mel_db.astype(np.float32))

# === Основной цикл ===
for fname in tqdm(os.listdir(input_dir)):
    if not fname.endswith(".mp3"):
        continue
    label = fname.split('_')[0]  # "sparrow_001.mp3" => "sparrow"
    fpath = os.path.join(input_dir, fname)
    process_audio(fpath, label)

print("✅ Обработка завершена! Все сегменты сохранены в", output_dir)


100%|██████████| 12/12 [00:25<00:00,  2.12s/it]

✅ Обработка завершена! Все сегменты сохранены в mel_data


In [2]:
import tensorflow as tf
import numpy as np
import glob
import os

def load_mel_dataset(root_dir):
    X, y = [], []
    labels = sorted(os.listdir(root_dir))
    label_to_idx = {lbl: i for i, lbl in enumerate(labels)}

    for lbl in labels:
        files = glob.glob(os.path.join(root_dir, lbl, "*.npy"))
        for f in files:
            mel = np.load(f)
            X.append(mel)
            y.append(label_to_idx[lbl])

    X = np.array(X)[..., np.newaxis]  # добавляем канал
    y = np.array(y)
    return X, y

X, y = load_mel_dataset("mel_data")
print(X.shape, y.shape)


(428, 64, 126, 1) (428,)
